# import

In [3]:
# EJThompson 02/12/24

## import functions from .py utils file: 
from Utilities.preprocessing import *


# set paths and params 

In [4]:
in_out = r'Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\raw_bpod_to_process\\'

# MAIN:

In [11]:


# reloads imports (utils) if something changes
%reload_ext autoreload
%autoreload 2

#### MAIN ####
for InputPath in os.listdir(in_out):
    if 'fig' in InputPath:
        continue
    
    InputPathCurrent = os.path.join(in_out, InputPath)
    
    if InputPathCurrent[-2] == 'a': #if its a .mat and not a fig
        if os.stat(InputPathCurrent).st_size > 200000: #more thann 200kb 
            if InputPathCurrent != '.DS_Store': #if file is not the weird hidden file 
                print(InputPathCurrent)
                Current_file = loadmat(InputPathCurrent)
                Behav_Data = Current_file
                Sessions = Sessions + 1
                File_dates = File_dates + [InputPathCurrent[-19:-4]]
    
    display(HTML(f"<b>Processing data for: {InputPath}</b>"))
    # InputPathCurrent = (InputPath + CurrentAnimal + r'\Sequence_Automated\Session Data\\')
    
    Save_path = os.path.join(in_out,InputPath.split('Sequence')[0] + InputPath.split('_')[-2])

    #Convert to python friendly format:
    convert_nested_structs(Behav_data)
    # Extract GUI info
    Trial_settings = todict(Behav_data['SessionData']['TrialSettings'][0])
    FinalRewardAmount = []
    for item in Behav_data['SessionData']['SessionVariables']['TLevel']:
        TLevel = item
        FinalRewardAmount = FinalRewardAmount + [Behav_data['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][4]]

    # save out training levels on their own
    filename = 'Preprocessed_TrainingLevels' 
    with open(Save_path + '\\'+ filename, 'wb') as fp:
        pickle.dump(Behav_data['SessionData']['SessionVariables']['TLevel'], fp)

    # save out led intensites and reward amounts on their own:
    LED_Intensities = pd.DataFrame({'Port2':Behav_data['SessionData']['SessionVariables']['LEDIntensitys']['port2'],
                            'Port3':Behav_data['SessionData']['SessionVariables']['LEDIntensitys']['port3'],
                            'Port4':Behav_data['SessionData']['SessionVariables']['LEDIntensitys']['port4'],
                            'Port5':Behav_data['SessionData']['SessionVariables']['LEDIntensitys']['port5']})
    LED_Intensities.to_csv(Save_path + '/Preprocessed_LED_Intensities.csv')
    RewardAmounts = pd.DataFrame({'Port1':Behav_data['SessionData']['SessionVariables']['RewardAmount']['port1'],
                                    'Port2':Behav_data['SessionData']['SessionVariables']['RewardAmount']['port2'],
                                    'Port3':Behav_data['SessionData']['SessionVariables']['RewardAmount']['port3'],
                                    'Port4':Behav_data['SessionData']['SessionVariables']['RewardAmount']['port4']})
    RewardAmounts.to_csv(Save_path + '/Preprocessed_RewardAmounts.csv')



    #Extract PortIn times for each port and check for errors (inside this function):
    All_PortIn_Times,All_PortOut_Times,All_Port_references = extract_poke_times(Behav_data)

    
    #remove nans (times when part [in or out poke] of the event was dropped for some reason by bpod)
    All_PortIn_Times_fixed,All_PortOut_Times_fixed ,All_Port_references_fixed = remove_dropped_in_events(All_PortIn_Times,All_PortOut_Times,All_Port_references)
    
    # Resort these in time:
    All_PortIn_Times_sorted,All_PortOut_Times_sorted,All_Port_references_sorted = time_sort(All_PortIn_Times,All_PortOut_Times,All_Port_references)
    
    #fix stupid error that suddently appeared
    if len(All_Port_references_sorted) == 1:
        All_Port_references_sorted = All_Port_references_sorted[0]
        All_PortIn_Times_sorted = All_PortIn_Times_sorted[0]
        All_PortOut_Times_sorted = All_PortOut_Times_sorted[0]
    #extract reward times:
    Reward_ts = extract_reward_times(Behav_data)

    # find reward inds and align rewarded ts to poke events:
    Rewarded_event_inds = find_reward_inds(All_PortIn_Times_sorted,All_Port_references_sorted,Reward_ts)
    Reward_ts = np.asarray(Reward_ts)
    Reward_ts = Reward_ts[np.logical_not(np.isnan(Reward_ts))]
    Reward_ts = list(Reward_ts)
    Reward_ts_aligned = align_trigger_to_index(Reward_ts,Rewarded_event_inds,All_Port_references_sorted)

    #extract trial start time stamps
    Trial_start_ts = extract_trial_timestamps(Behav_data)

    #extract trial end times:
    Trial_end_ts = extract_trial_end_times(Behav_data)

    #determine trial IDs
    trial_id = determine_trial_id(All_PortIn_Times_sorted,Trial_end_ts)

    # align trial start and end times to poke events
    trialstart_index = find_trialstart_index(trial_id)
    trial_start_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_start_ts)
    trial_end_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_end_ts)

    # optostim data:
    if Trial_settings['GUI']['OptoStim'] == 1: 
        ## save out opto settings as a dataframe
        Opto_StimPoke = Trial_settings['GUI']['StimPoke']
        Opto_PulsePower = Trial_settings['GUI']['PulsePower']
        Opto_OptoChance = Trial_settings['GUI']['OptoChance']
        Opto_PulseDuration = Trial_settings['GUI']['PulseDuration']
        Opto_PulseInterval = Trial_settings['GUI']['PulseInterval']
        Opto_TrainDuration = Trial_settings['GUI']['TrainDuration']
        #create opto dataframe
        Opto_settings = pd.DataFrame(
        {'StimPoke' : [Opto_StimPoke],
            'PulsePower':[Opto_PulsePower],
            'OptoChance':[Opto_OptoChance],
            'PulseDuration' : [Opto_PulseDuration],
            'PulseInterval':[Opto_PulseInterval],
            'TrainDuration':[Opto_TrainDuration]})
        #save
        Opto_settings.to_csv(Save_path + '/Opto_settings.csv')

        #pull out optotrials from data
        optotrials = Behav_data['SessionData']['SessionVariables']['OptoStim']
        #align these to dataframe:
        executed_optotrials = optotrials[0:trial_id[-1]]
        optotrials_aligned = align_opto_trials_to_dataframe(trial_id,executed_optotrials)
    else:
        #no optostim so fill this column with NaNs
        optotrials_aligned = ['NaN'] * len(trial_id)

    #determine LED and reward states for each trial and align them to trials:
    IntermediateRewards = []
    LED_intensities = []
    for item in Behav_data['SessionData']['SessionVariables']['TLevel']:
        TLevel = item
        IntermediateRewards = IntermediateRewards + [list(Behav_data['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][0:4])]
        LED_intensities = LED_intensities + [list(Behav_data['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][6:10])]
    aligned_LED_intensities = align_trial_start_end_timestamps(trial_id,trialstart_index,LED_intensities)
    aligned_IntermediateRewards = align_trial_start_end_timestamps(trial_id,trialstart_index,IntermediateRewards)

    # #Search for timestamps for given animal and session 
    # TimeStampsExist, TimeStampPath = FindTimestamps(filedate,CameraPath,CurrentAnimal)

    # if TimeStampsExist:
    #     print('Timestamps Found')
    #     #Load camera timestamps:
    #     Camera_ts_raw = load_camera_timestamps(TimeStampPath)
    #     #Convert to seconds and uncycle:
    #     Camera_ts = convert_uncycle_Timestamps(Camera_ts_raw)
    #     #check for dropped frames:
    #     check_timestamps(Camera_ts, Frame_rate = 40)
    #     # Find triggers:
    #     Camera_trig_states = find_trigger_states(Camera_ts_raw)
    #     #check if triggers are working:
    #     result = np.max(Camera_trig_states) == np.min(Camera_trig_states)
    #     if not result:
    #         # make camera dataframe:
    #         Camera_dataframe = pd.DataFrame(
    #             {'Time Stamps': Camera_ts,
    #             'Trigger State': Camera_trig_states,
    #             'DataPath': ([TimeStampPath] * len(Camera_ts))})

    #         #Save Data
    #         Camera_dataframe.to_csv(Save_path + '/PreProcessed_CameraData.csv')

    #         #find camera inds that align with trigger start and stop events (trial start and first poke):
    #         Trial_start_camera_inds,Poke1_camera_inds = Find_TrialStart_and_Poke1_camera_inds(Camera_trig_states)
    #         #align behavioural data (trial starts) with camera timestamps 
    #         Trial_start_Camera_Ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Camera_ts[Trial_start_camera_inds])
            
            

    #         #align behavioural data (trial ends) with camera timestamps 
    #         Trial_end_Camera_Ts_aligned = generate_aligned_trial_end_camera_ts(Trial_start_camera_inds,trial_id,trialstart_index,Camera_ts)
    #         #align behavioural data (first poke in port1) with camera timestamps 
    #         First_poke_Camera_Ts_aligned = align_firstpoke_camera_timestamps(trial_id,trialstart_index,Camera_ts[Poke1_camera_inds],All_Port_references_sorted)
        
    #     else:
    #         print('Triggers Broken')
    #         TimeStampsExist = False
    #         Trial_start_Camera_Ts_aligned = ['NaN'] * len(trial_id)
    #         Trial_end_Camera_Ts_aligned = ['NaN'] * len(trial_id)
    #         First_poke_Camera_Ts_aligned = ['NaN'] * len(trial_id)
    # else:
    Trial_start_Camera_Ts_aligned = ['NaN'] * len(trial_id)
    Trial_end_Camera_Ts_aligned = ['NaN'] * len(trial_id)
    First_poke_Camera_Ts_aligned = ['NaN'] * len(trial_id)

    ## align Training level for each trial:
    Training_Levels = align_opto_trials_to_dataframe(trial_id,Behav_data['SessionData']['SessionVariables']['TLevel'])

    # make portin dataframe:
    PortIn_df = pd.DataFrame(
        {'Trial_id' : trial_id,
            'Trial_Start' : trial_start_ts_aligned,
            'Port': All_Port_references_sorted,
            'PokeIn_Time': All_PortIn_Times_sorted,
            'PokeOut_Time': All_PortOut_Times_sorted,
            'Reward_Times': Reward_ts_aligned,
            'Trial_End' : trial_end_ts_aligned,
            'Trial_Start_Camera_Time':Trial_start_Camera_Ts_aligned,
            'Trial_End_Camera_Time':Trial_end_Camera_Ts_aligned,
            'First_poke_camera_timestamp': First_poke_Camera_Ts_aligned,
            'Port 2,3,4,5 LED intensities': aligned_LED_intensities,
            'Port 1,2,3,4 RewardAmount':aligned_IntermediateRewards,
            'OptoCondition':optotrials_aligned,
        'TrainingLevel': Training_Levels})
    
    #Save Data
    PortIn_df.to_csv(Save_path +'/PreProcessed_RawPokeData.csv')

    #PART2##########################Transitions:

    #Determine Transition times and types for all events 
    ##old way:
#             Transition_times, Transition_types,transition_reference_time = Determine_Transition_Times_and_Types(All_PortIn_Times_sorted , All_Port_references_sorted)
    out_in_Transition_times, in_in_Transition_times, Transition_types, out_in_transition_reference,in_in_transition_reference = Determine_Transition_Times_and_Types(All_PortIn_Times_sorted ,All_PortOut_Times_sorted, All_Port_references_sorted)
    
    
    #Split transtion types into first and last ports: 
    start_port_ids = Start_End_port_id(Transition_types,0)
    end_port_ids = Start_End_port_id(Transition_types,1)
    #Align start and end port times
    End_Port_in_time = All_PortIn_Times_sorted[1::]
    Start_Port_in_time = All_PortIn_Times_sorted[0:-1]
    End_Port_out_time = All_PortOut_Times_sorted[1::]
    Start_Port_out_time = All_PortOut_Times_sorted[0:-1]
    #Find Port repeat events (double pokes)
    Non_Port_repeat = determine_RepeatPort_events(start_port_ids,end_port_ids)
    #Determine which transitions are good: less than 1.5s
    out_in_Filtered_transitions = filter_transitons_by_latency(out_in_Transition_times, 2)
    in_in_Filtered_transitions = filter_transitons_by_latency(in_in_Transition_times, 2)

    # if TimeStampsExist:
    #     # Align camera ts to each transition event 
    #     ## calculate from each time stamp
    #     first_port_camera_ts = port_events_in_camera_time(trial_start_ts_aligned,Start_Port_in_time,Trial_start_Camera_Ts_aligned)
    #     second_port_camera_ts = first_port_camera_ts + in_in_Transition_times
    # else:
    first_port_camera_ts = ['NaN'] * len(Transition_types)
    second_port_camera_ts = ['NaN'] * len(Transition_types)

    # make dataframe:
    Transition_df = pd.DataFrame(
        {'Trial_id' : trial_id[0:-1],
        'Transition_type' : Transition_types,
            'Start_Port':start_port_ids, 
            'End_Port':end_port_ids,
            'Start_Poke_in_time': Start_Port_in_time,
            'Start_Poke_out_time': Start_Port_out_time,
            'End_Poke_in_time': End_Port_in_time,
            'End_Poke_out_time': End_Port_out_time,
            'out_in_Latency' : out_in_Transition_times,
            'in_in_Latency': in_in_Transition_times,
            'First_port_in_camera_time':first_port_camera_ts,
            'Second_port_in_camera_time':second_port_camera_ts,
            'Repeat_Filter':Non_Port_repeat,
            '2s_Time_Filter_out_in': out_in_Filtered_transitions,
            '2s_Time_Filter_in_in': in_in_Filtered_transitions,
            'OptoCondition':optotrials_aligned[0:-1],
            'TrainingLevel': Training_Levels[0:-1],
            'Port 2,3,4,5 LED intensities': aligned_LED_intensities[0:-1],
            'Port 1,2,3,4 RewardAmount':aligned_IntermediateRewards[0:-1]})
    #Save Data
    Transition_df.to_csv(Save_path + '/PreProcessed_TransitionData.csv')

    #PART3#########################Sequences:

    # Define useful port/seq related information
    port1 = Behav_data['SessionData']['TrialSequence'][0][0]
    port2 = Behav_data['SessionData']['TrialSequence'][0][1]
    port3 = Behav_data['SessionData']['TrialSequence'][0][2]
    port4 = Behav_data['SessionData']['TrialSequence'][0][3]
    port5 = Behav_data['SessionData']['TrialSequence'][0][4]
    sequence1 = int(str(port1) + str(port2))
    sequence2 = int(str(port2) + str(port3))
    sequence3 = int(str(port3) + str(port4))
    sequence4 = int(str(port4) + str(port5))

    #Filter transitons into sequences for each port by finding port and then transitions that happen after that port which are in a string that satisfies the transition filter time
    # there is now a lower filter - ie. any sequences that are shorter than 0.05 are discounted:
    ## these are from in to in as we dont care about transiton times, just poke in times for eahc port 
    Port1_Time_Filtered_seq_ids,Port1_Time_Filtered_seq_times,Port1_Reference_times = CreateSequences_TimeandPort(Transition_types,in_in_Transition_times,port1,in_in_transition_reference,Transition_filter_time = 2.0)
    Port2_Time_Filtered_seq_ids,Port2_Time_Filtered_seq_times,Port2_Reference_times = CreateSequences_TimeandPort(Transition_types,in_in_Transition_times,port2,in_in_transition_reference,Transition_filter_time = 2.0)
    Port3_Time_Filtered_seq_ids,Port3_Time_Filtered_seq_times,Port3_Reference_times = CreateSequences_TimeandPort(Transition_types,in_in_Transition_times,port3,in_in_transition_reference,Transition_filter_time = 2.0)
    Port4_Time_Filtered_seq_ids,Port4_Time_Filtered_seq_times,Port4_Reference_times = CreateSequences_TimeandPort(Transition_types,in_in_Transition_times,port4,in_in_transition_reference,Transition_filter_time = 2.0)
    Port5_Time_Filtered_seq_ids,Port5_Time_Filtered_seq_times,Port5_Reference_times = CreateSequences_TimeandPort(Transition_types,in_in_Transition_times,port5,in_in_transition_reference,Transition_filter_time = 2.0)

    #Filter transitons into sequences that are within the transition filter time (not filtered to start at first poke):
    Time_Filtered_seq_ids,Time_Filtered_seq_times,Reference_times = CreateSequences_Time(Transition_types,out_in_Transition_times,port1,in_in_transition_reference,Transition_filter_time = 2.0)

    # make dataframes:
    Sequence_df_timefiltered_port1aligned = pd.DataFrame(
        {'Sequence_ids' : Port1_Time_Filtered_seq_ids,
            'Sequence_times':Port1_Time_Filtered_seq_times,
            'SessionTime_Reference':Port1_Reference_times})
    Sequence_df_timefiltered_port2aligned = pd.DataFrame({
            'Sequence_ids' : Port2_Time_Filtered_seq_ids,
            'Sequence_times':Port2_Time_Filtered_seq_times,
            'SessionTime_Reference':Port2_Reference_times})
    Sequence_df_timefiltered_port3aligned = pd.DataFrame({    
            'Sequence_ids' : Port3_Time_Filtered_seq_ids,
            'Sequence_times':Port3_Time_Filtered_seq_times,
            'SessionTime_Reference':Port3_Reference_times})
    Sequence_df_timefiltered_port4aligned = pd.DataFrame({     
            'Sequence_ids' : Port4_Time_Filtered_seq_ids,
            'Sequence_times':Port4_Time_Filtered_seq_times,
            'SessionTime_Reference':Port4_Reference_times})
    Sequence_df_timefiltered_port5aligned = pd.DataFrame({
            'Sequence_ids' : Port5_Time_Filtered_seq_ids,
            'Sequence_times':Port5_Time_Filtered_seq_times,
            'SessionTime_Reference':Port5_Reference_times})

    Sequence_timefilteredonly_df = pd.DataFrame(
        {'Sequence_ids' : Time_Filtered_seq_ids,
            'Sequence_times':Time_Filtered_seq_times,
            'SessionTime_Reference':Reference_times})

    #Save Data
    Sequence_df_timefiltered_port1aligned.to_csv(Save_path + '/PreProcessed_Sequence_df_timefiltered_port1aligned.csv')
    Sequence_df_timefiltered_port2aligned.to_csv(Save_path + '/PreProcessed_Sequence_df_timefiltered_port2aligned.csv')
    Sequence_df_timefiltered_port3aligned.to_csv(Save_path + '/PreProcessed_Sequence_df_timefiltered_port3aligned.csv')
    Sequence_df_timefiltered_port4aligned.to_csv(Save_path + '/PreProcessed_Sequence_df_timefiltered_port4aligned.csv')
    Sequence_df_timefiltered_port5aligned.to_csv(Save_path + '/PreProcessed_Sequence_df_timefiltered_port5aligned.csv')

    Sequence_timefilteredonly_df.to_csv(Save_path + '/PreProcessed_Sequence_timefilteredonly_df.csv')

    #Make final session information dataframe
    TrainingLevels = list(Trial_settings['GUIMeta']['TrainingLevel']['String'])
    SessionLevel = Trial_settings['GUI']['TrainingLevel']
    no_rewarded_events = number_of_rewarded_events(Reward_ts_aligned)
    
    #experiment or training session:
    if Trial_settings['GUI']['ExperimentType'] == 2:
        Experiment = 1
    else:
        Experiment = 0

    Session_information = pd.DataFrame(
        {'Port1' : [port1],
            'Port2':[port2],
            'Port3':[port3],
            'Port4':[port4],
            'Port5':[port5],
        'Transition1':sequence1,
        'Transition2':sequence2,
        'Transition3':sequence3,
        'Transition4':sequence4,
        'n_Trials': trial_id[-1],
        'n_rewards': no_rewarded_events,
        'FinalRewardAmount': [FinalRewardAmount],
        'SessionLevel':[TLevel],
        'Experiment':Experiment,
        'CameraData': TimeStampsExist,})

    #Save Data
    Session_information.to_csv(Save_path + '/PreProcessed_SessionInfo.csv')
    Processed = (Processed + str(Session) +',')
else:
    Skipped = (Skipped + str(Session) +',')
    if len(Skipped) == 0:
        Skipped == 'None'
    if len(Processed) == 0:
        Processed == 'None'
    print('Already Processed so skipped: ' + Skipped)
    print('Processed: ' + Processed)

print('finished')

Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\raw_bpod_to_process\\AP5_2_R_Sequence_Automated_20241117_101411.mat


KeyboardInterrupt: 